# Pandas Workflow com LlamaIndex Workflows

>Este notebook demonstra como criar um sistema de análise de dados usando LlamaIndex, Pandas e Groq para consultas em linguagem natural.


## Sobre a Atualização do LlamaIndex

O **LlamaIndex** descontinuou o módulo **QueryPipeline** em favor de uma nova abordagem chamada **Workflows**.
Essa mudança foi implementada na versão `0.11` do LlamaIndex, com o objetivo de oferecer uma arquitetura mais
flexível e escalável para a construção de aplicações de IA generativa.

### Por que o QueryPipeline foi descontinuado?
O QueryPipeline era uma API declarativa útil para orquestrar consultas simples a avançadas sobre os dados.
Porém, ele se mostrou limitado para cenários mais dinâmicos. Por isso, os **Workflows** foram introduzidos,
trazendo uma arquitetura orientada a eventos para fluxos de trabalho mais sofisticados.

### O que são Workflows?
Workflows permitem orquestrar etapas personalizadas de forma assíncrona e condicional, facilitando
integrações complexas e manipulação de dados em tempo real.

**Documentação oficial:** [docs.llamaindex.ai](https://docs.llamaindex.ai)



#Código:


##1 Instalar Dependências:

In [1]:
# Instalar dependências (-q para instalação silenciosa)
!pip install -q jedi>=0.16, llama-index llama-index-llms-openai llama-index-llms-groq llama-index-experimental gradio fpdf

##1.1 Para uso no Colab


In [2]:
import nest_asyncio
nest_asyncio.apply()

##2 Imports

In [3]:
import pandas as pd
import textwrap
import re
from pydantic import BaseModel, Field, field_validator, ConfigDict, ValidationError
from llama_index.core import Settings
from llama_index.llms.groq import Groq
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.core.workflow import Workflow, Event, StartEvent as BaseStartEvent, StopEvent, step
from llama_index.core.prompts import PromptTemplate
from google.colab import userdata
from typing import Any, List, Tuple, Optional
import asyncio
import gradio as gr
from fpdf import FPDF
from datetime import datetime

##3 Instruções e Prompts

(Importante para validar a saída, pode ser usado como debug ou  )

In [4]:
# ===== INSTRUÇÕES PARA CONVERSÃO DE CONSULTAS =====
instruction_str = (
    "1. Converta a consulta para código Python executável usando Pandas.\n"
    "2. A linha final do código deve ser uma expressão Python que possa ser chamada com a função `eval()`.\n"
    "3. O código deve representar uma solução para a consulta.\n"
    "4. IMPRIMA APENAS A EXPRESSÃO FINAL.\n"
    "5. Não coloque a expressão entre aspas.\n"
    "6. Evite atribuições (=) na linha final - prefira expressões que retornem valores.\n"
    "7. Para operações de múltiplas linhas, termine com uma expressão que retorne o resultado.\n"
    "8. Exemplos válidos:\n"
    "   - df.groupby('coluna')['valor'].sum()\n"
    "   - df['coluna'].value_counts().head(5)\n"
    "   - df.describe()\n"
    "9. Evite códigos como 'df['coluna'] = valor' - prefira consultas que retornem dados.\n"
)

# ===== PROMPTS =====
pandas_prompt_str = (
    "Você está trabalhando com um dataframe do pandas em Python chamado `df`.\n"
    "{colunas_detalhes}\n\n"
    "Este é o resultado de `print(df.head())`:\n"
    "{df_str}\n\n"
    "Siga estas instruções:\n"
    "{instruction_str}\n"
    "Consulta: {query_str}\n\n"
    "Expressão:"
)

RESPONSE_SYNTHESIS_PROMPT_STR = (
   "Dada uma pergunta de entrada, atue como analista de dados e elabore uma resposta a partir dos resultados da consulta.\n"
   "Responda de forma natural, sem introduções como 'A resposta é:' ou algo semelhante.\n"
   "Consulta: {query_str}\n\n"
   "Instruções do Pandas (opcional):\n{pandas_instructions}\n\n"
   "Saída do Pandas: {pandas_output}\n\n"
   "Resposta: "
   "Ao final, exibir o código usado para gerar a resposta, no formato: O código utilizado foi {pandas_instructions}"
)

print("✅ Prompts configurados com sucesso!")

✅ Prompts configurados com sucesso!


##4 Configuração Pydantic V2

In [16]:
class LLMConfig(BaseModel):
    model: str = Field(..., description="Nome do modelo Groq a ser usado")
    api_key: str = Field(..., description="Chave da API Groq")
    data_url: str = Field(..., description="URL do CSV com os dados")

    @field_validator("data_url")
    @classmethod
    def validar_url(cls, v: str) -> str:
        if not (v.startswith("http://") or v.startswith("https://")):
            raise ValueError("data_url deve começar com http:// ou https://")
        return v

    @field_validator("api_key")
    @classmethod
    def validar_api_key(cls, v: str) -> str:
        if not v or len(v.strip()) == 0:
            raise ValueError("api_key não pode ser vazia")
        return v

print("✅ Configuração LLM com Pydantic criada!")


# ===== HISTÓRICO DE PERGUNTAS/RESPOSTAS =====
class Historico(BaseModel):
    entradas: List[Tuple[str, str]] = []

    def adicionar(self, pergunta: str, resposta: str):
        """Adiciona uma entrada ao histórico se pergunta e resposta forem válidas."""
        if pergunta and resposta:
            self.entradas.append((pergunta, resposta))

print("✅ Modelo de histórico com Pydantic criado!")

✅ Configuração LLM com Pydantic criada!
✅ Modelo de histórico com Pydantic criado!


##5 Modelos de Eventos

(Rastrear cada passo da interação, como um histórico organizado)

In [27]:
# --- Modelos de Eventos com Pydantic ---

class CodeEvent(Event):
    pandas_prompt: str
    query: str
    df: pd.DataFrame
    model_config = ConfigDict(arbitrary_types_allowed=True)

class OutputEvent(Event):
    pandas_code: str
    pandas_output: object = None
    query: str
    pandas_prompt: str
    df: pd.DataFrame
    model_config = ConfigDict(arbitrary_types_allowed=True)

class ExecutedEvent(Event):
    pandas_code: str
    pandas_output: object = None
    query: str
    pandas_prompt: str
    df: pd.DataFrame
    model_config = ConfigDict(arbitrary_types_allowed=True)

print("✅ Eventos com DataFrame propagado corrigidos!")

✅ Eventos com DataFrame propagado corrigidos!


##6 Funções Auxiliares

In [29]:
def carregar_arquivo(caminho_arquivo: str, df_estado: pd.DataFrame = None):
    """
    Carrega um CSV do caminho fornecido. Se caminho_arquivo for None, mantém df_estado.
    Retorna mensagem de status e DataFrame atualizado.
    """
    if caminho_arquivo is None or caminho_arquivo == "":
        return "Nenhum arquivo enviado. Usando dados atuais.", df_estado
    try:
        df_novo = pd.read_csv(caminho_arquivo)
        return "Arquivo carregado com sucesso!", df_novo
    except Exception as e:
        return f"Erro ao carregar arquivo: {str(e)}", df_estado

def formatar_texto(response: str, largura: int = 100, imprimir: bool = True):
    texto_formatado = textwrap.fill(response, width=largura)
    if imprimir:
        print(texto_formatado)
    return texto_formatado

def descricao_colunas(df: pd.DataFrame) -> str:
    descricao = "\n".join([f"`{col}`: {str(df[col].dtype)}" for col in df.columns])
    return "Colunas do DataFrame:\n" + descricao

def limpar_codigo_pandas(codigo: str) -> str:
    """Limpa e valida o código Pandas gerado pela LLM"""
    codigo = re.sub(r'```(?:python)?\n?', '', codigo)
    linhas = codigo.split('\n')
    codigo_limpo = []

    for linha in linhas:
        linha = linha.strip()
        if (linha and
            not linha.startswith('#') and
            not linha.startswith('//') and
            not linha.lower().startswith('resposta:') and
            not linha.lower().startswith('resultado:') and
            not linha.lower().startswith('código:')):
            codigo_limpo.append(linha)

    codigo_final = '\n'.join(codigo_limpo).strip()

    # Se há múltiplas linhas, pega a última linha como expressão final
    if '\n' in codigo_final:
        linhas_codigo = [l.strip() for l in codigo_final.split('\n') if l.strip()]
        # Pega a última linha que seja uma expressão (não atribuição)
        for linha in reversed(linhas_codigo):
            if not ('=' in linha and not linha.startswith('df[')):
                codigo_final = linha
                break

    return codigo_final

print("✅ Funções auxiliares configuradas!")

✅ Funções auxiliares configuradas!


##7 Configuração da API e Dados

In [30]:
# Configure sua chave da API Groq nos Secrets do Colab
# Vá em:  (ícone da chave) -> Add new secret
# Name: Groq_API
# Value: sua_chave_aqui

try:
    key = userdata.get("Groq_API")
    print("✅ Chave API carregada com sucesso!")
except Exception as e:
    print("❌ Erro ao carregar chave API:", e)
    print("Configure a chave 'Groq_API' nos Secrets do Colab")

# ===== CONFIGURAÇÃO E DADOS =====
config = LLMConfig(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    api_key=key,
    data_url="https://raw.githubusercontent.com/YuriArduino/Estudos_Artificial_Intelligence/refs/heads/Dados/vendas.csv"
)

df = pd.read_csv(config.data_url)
Settings.llm = Groq(model=config.model, api_key=config.api_key)

print(" Estrutura do DataFrame:")
print(df.head())
print(f"\n Colunas disponíveis:")
print(df.columns.tolist())
print(f"\n Info do DataFrame:")
print(df.info())
print(f"\n✅ Configuração completa! Dataset: {df.shape[0]} linhas, {df.shape[1]} colunas")

✅ Chave API carregada com sucesso!
 Estrutura do DataFrame:
     ID_compra filial       cidade tipo_cliente     genero       tipo_produto  \
0  750-67-8428      A  Santo André       Membro   Feminino     Saúde e Beleza   
1  226-31-3081      C  São Caetano       Normal   Feminino        Eletrônicos   
2  631-41-3108      A  Santo André       Normal  Masculino               Casa   
3  123-19-1176      A  Santo André       Membro  Masculino     Saúde e Beleza   
4  373-73-7910      A  Santo André       Normal  Masculino  Esportes e Viagem   

   preco_unitario  quantidade  imposto_5%     total        data      hora  \
0           74.69           7     26.1415  548.9715  2024-01-05  13:08:00   
1           15.28           5      3.8200   80.2200  2024-03-08  10:29:00   
2           46.33           7     16.2155  340.5255  2024-03-03  13:23:00   
3           58.22           8     23.2880  489.0480  2024-01-27  20:33:00   
4           86.31           7     30.2085  634.3785  2024-02-08  10:

##8 Classe Workflow Principal

In [31]:
# --- Workflow Compatível com Diferentes Versões ---
from llama_index.core.workflow import Workflow, step, StartEvent, StopEvent
import pandas as pd

class PandasWorkflow(Workflow):

    @step(pass_context=True)  # Adiciona pass_context para compatibilidade
    async def iniciar_processamento(self, ctx, ev: StartEvent) -> CodeEvent:
        """
        Step que GARANTE receber StartEvent
        """
        print(f" StartEvent recebido no workflow!")
        print(f"📝 Tipo do evento: {type(ev)}")
        print(f"📋 Conteúdo: {dir(ev)}")

        # Múltiplas formas de extrair dados para máxima compatibilidade
        query = None
        df_local = None

        # Método 1: Atributos diretos
        if hasattr(ev, 'query'):
            query = ev.query
            df_local = ev.df
            print("✅ Dados extraídos via atributos diretos")

        # Método 2: get() se disponível
        elif hasattr(ev, 'get'):
            query = ev.get('query')
            df_local = ev.get('df')
            print("✅ Dados extraídos via get()")

        # Método 3: Dicionário
        elif hasattr(ev, '__getitem__'):
            try:
                query = ev['query']
                df_local = ev['df']
                print("✅ Dados extraídos via índice")
            except:
                pass

        # Método 4: Contexto do workflow
        if query is None and hasattr(ctx, 'data'):
            query = ctx.data.get('query')
            df_local = ctx.data.get('df')
            print("✅ Dados extraídos via contexto")

        # Fallback: usar dados globais se nada funcionar
        if query is None:
            print("⚠️ Usando fallback para dados")
            query = getattr(self, '_temp_query', 'Consulta padrão')
            df_local = getattr(self, '_temp_df', pd.DataFrame())

        print(f"📊 Query final: {query}")
        print(f"📊 DataFrame shape: {df_local.shape if df_local is not None else 'None'}")

        # Processa o prompt
        colunas_info = descricao_colunas(df_local)
        prompt_text = pandas_prompt_str.format(
            colunas_detalhes=colunas_info,
            df_str=df_local.head(5),
            instruction_str=instruction_str,
            query_str=query
        )

        return CodeEvent(
            pandas_prompt=prompt_text,
            query=query,
            df=df_local
        )

    @step
    async def gerar_codigo(self, ev: CodeEvent) -> OutputEvent:
        """Gera código pandas"""
        print(f"🔧 Gerando código para: {ev.query}")

        response = await Settings.llm.acomplete(ev.pandas_prompt)
        codigo_bruto = str(response).strip()
        codigo_limpo = limpar_codigo_pandas(codigo_bruto)
        print(f"✅ Código: {codigo_limpo}")

        return OutputEvent(
            pandas_code=codigo_limpo,
            pandas_output=None,
            query=ev.query,
            pandas_prompt=ev.pandas_prompt,
            df=ev.df
        )

    @step
    async def executar_codigo(self, ev: OutputEvent) -> ExecutedEvent:
        """Executa o código"""
        print(f"▶️ Executando código: {ev.pandas_code}")

        try:
            contexto = {"df": ev.df, "pd": pd, "__builtins__": __builtins__}
            resultado = eval(ev.pandas_code, contexto)
            print(f"✅ Resultado: {resultado}")
        except Exception as e:
            resultado = f"Erro: {str(e)}"
            print(f"❌ {resultado}")

        return ExecutedEvent(
            pandas_code=ev.pandas_code,
            pandas_output=resultado,
            query=ev.query,
            pandas_prompt=ev.pandas_prompt,
            df=ev.df
        )

    @step
    async def finalizar(self, ev: ExecutedEvent) -> StopEvent:
        """Finaliza com resposta"""
        print(f" Finalizando processamento")

        if isinstance(ev.pandas_output, str) and "Erro" in ev.pandas_output:
            resposta_final = f"Não foi possível processar: {ev.pandas_output}"
        else:
            prompt_text = RESPONSE_SYNTHESIS_PROMPT_STR.format(
                query_str=ev.query,
                pandas_instructions=ev.pandas_code,
                pandas_output=ev.pandas_output
            )
            resposta = await Settings.llm.acomplete(prompt_text)
            resposta_final = str(resposta).strip()

        return StopEvent(result={
            "resposta_final": resposta_final,
            "pandas_code": ev.pandas_code,
            "pandas_output": ev.pandas_output,
            "query": ev.query
        })

print("✅ Workflow robusto definido!")

✅ Workflow robusto definido!


##9 Função de Execução

In [32]:
async def executar_consulta(query: str, df_local: pd.DataFrame, mostrar_detalhes: bool = True):
    """
    Versão robusta que funciona com diferentes configurações
    """
    try:
        print(f" Iniciando processamento: {query}")

        # Tentativa 1: Workflow padrão
        try:
            wf = PandasWorkflow()

            # Armazena dados temporariamente no workflow para fallback
            wf._temp_query = query
            wf._temp_df = df_local

            # Tenta diferentes formas de criar StartEvent
            try:
                # Método 1: StartEvent com atributos
                start_event = StartEvent()
                start_event.query = query
                start_event.df = df_local
                result = await wf.run(start_event)
            except:
                # Método 2: run com parâmetros diretos
                result = await wf.run(query=query, df=df_local)

            print(f"✅ Workflow executado com sucesso!")

            if hasattr(result, 'result'):
                dados = result.result

                if mostrar_detalhes:
                    print("\n === RESULTADO ===")
                    print(f"Query: {dados.get('query')}")
                    print(f"Código: {dados.get('pandas_code')}")
                    print(f"Output: {dados.get('pandas_output')}")
                    print(f"Resposta: {dados.get('resposta_final')}")

                return type('WorkflowResult', (), dados)()

        except Exception as e:
            print(f"⚠️ Workflow falhou: {e}")
            raise

    except Exception as e:
        print(f"❌ Todas as tentativas falharam: {e}")
        print("🔄 Tentando processamento direto...")

        # Fallback: processamento direto sem workflow
        return await processar_consulta_direta(query, df_local)

# Função de processamento direto (fallback)
async def processar_consulta_direta(query: str, df_local: pd.DataFrame):
    """Processamento sem workflow como fallback"""
    try:
        print(" Processamento direto iniciado...")

        # Gerar código
        colunas_info = descricao_colunas(df_local)
        prompt_text = pandas_prompt_str.format(
            colunas_detalhes=colunas_info,
            df_str=df_local.head(5),
            instruction_str=instruction_str,
            query_str=query
        )

        response = await Settings.llm.acomplete(prompt_text)
        codigo_limpo = limpar_codigo_pandas(str(response).strip())
        print(f"✅ Código: {codigo_limpo}")

        # Executar código
        try:
            contexto = {"df": df_local, "pd": pd, "__builtins__": __builtins__}
            resultado = eval(codigo_limpo, contexto)
            print(f"✅ Resultado: {resultado}")
        except Exception as e:
            resultado = f"Erro na execução: {str(e)}"

        # Sintetizar resposta
        if isinstance(resultado, str) and "Erro" in resultado:
            resposta_final = f"Não foi possível processar: {resultado}"
        else:
            prompt_synthesis = RESPONSE_SYNTHESIS_PROMPT_STR.format(
                query_str=query,
                pandas_instructions=codigo_limpo,
                pandas_output=resultado
            )
            resposta = await Settings.llm.acomplete(prompt_synthesis)
            resposta_final = str(resposta).strip()

        return type('DirectResult', (), {
            'resposta_final': resposta_final,
            'pandas_code': codigo_limpo,
            'pandas_output': resultado,
            'query': query
        })()

    except Exception as e:
        return type('ErrorResult', (), {
            'resposta_final': f"❌ Erro no processamento: {str(e)}",
            'pandas_code': '',
            'pandas_output': None,
            'query': query
        })()

print("✅ Funções robustas definidas!")

async def executar_multiplas_consultas(consultas: list, df_local: pd.DataFrame = None):
    """
    Executa várias consultas em sequência usando um DataFrame opcional.
    """
    resultados = []

    for i, consulta in enumerate(consultas, 1):
        print(f"\n{'='*60}")
        print(f"CONSULTA {i}/{len(consultas)}")
        print(f"{'='*60}")

        resultado = await executar_consulta(consulta, df_local=df_local)
        resultados.append(resultado)

        print("\n")  # Espaçamento entre consultas

    return resultados

print("✅ Funções de execução adaptadas para DataFrame dinâmico!")


✅ Funções robustas definidas!
✅ Funções de execução adaptadas para DataFrame dinâmico!


9.2 Funções auxiliares Gradio

In [33]:
# --- Função para carregar CSV ---
def carregar_dados(file, df_estado):
    if file is None:
        return "❌ Nenhum arquivo selecionado", df_estado
    try:
        df_local = pd.read_csv(file)
        return f"✅ Arquivo carregado com sucesso! {df_local.shape[0]} linhas, {df_local.shape[1]} colunas.", df_local
    except Exception as e:
        return f"❌ Erro ao carregar arquivo: {str(e)}", df_estado


# --- Função para executar consulta com workflow ---
async def executar_consulta_async(pergunta, df_local):
    """
    Função assíncrona simplificada para o Gradio
    """
    try:
        resultado = await executar_consulta(pergunta, df_local, mostrar_detalhes=False)

        if resultado is not None:
            return resultado
        else:
            return type('MockResult', (), {
                'resposta_final': "❌ Erro: Workflow não executou corretamente",
                'pandas_code': '',
                'pandas_output': None,
                'query': pergunta
            })()

    except Exception as e:
        print(f"❌ Erro na função async: {str(e)}")
        import traceback
        traceback.print_exc()
        return type('MockResult', (), {
            'resposta_final': f"❌ Erro: {str(e)}",
            'pandas_code': '',
            'pandas_output': None,
            'query': pergunta
        })()

    except Exception as e:
        print(f"❌ Erro no workflow: {str(e)}")
        # Retorna um objeto mock em caso de erro
        return type('MockResult', (), {
            'resposta_final': f"❌ Erro ao processar consulta: {str(e)}",
            'pandas_code': '',
            'pandas_output': None,
            'query': pergunta
        })()


# --- Função para processar perguntas (versão síncrona para Gradio) ---
def processar_pergunta(pergunta, df_local, historico: Historico):
    if df_local is None:
        return "❌ Nenhum arquivo CSV carregado.", historico

    if not pergunta or not pergunta.strip():
        return "❌ Digite uma pergunta válida.", historico

    try:
        # Executa a função assíncrona do workflow
        resultado = asyncio.run(executar_consulta_async(pergunta, df_local))
        resposta = resultado.resposta_final if hasattr(resultado, 'resposta_final') else "❌ Erro ao processar a consulta."

        # Atualiza o histórico usando Pydantic
        historico.adicionar(pergunta, resposta)

        return resposta, historico

    except Exception as e:
        erro_msg = f"❌ Erro ao processar pergunta: {str(e)}"
        historico.adicionar(pergunta, erro_msg)
        return erro_msg, historico


# --- Função para gerar PDF ---
def gerar_pdf(historico: Historico):
    if not historico.entradas:
        return "❌ Nenhum dado para adicionar ao PDF"

    try:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        caminho_pdf = f"relatorio_pandas_{timestamp}.pdf"

        pdf = FPDF()
        pdf.add_page()
        pdf.set_auto_page_break(auto=True, margin=15)

        # Título do relatório
        pdf.set_font("Arial", "B", 16)
        pdf.cell(0, 10, "Relatório de Consultas Pandas", ln=True, align="C")
        pdf.ln(10)

        # Adiciona cada pergunta e resposta
        for i, (pergunta, resposta) in enumerate(historico.entradas, 1):
            # Pergunta
            pdf.set_font("Arial", "B", 12)
            pdf.multi_cell(0, 8, f"Pergunta {i}: {pergunta}")
            pdf.ln(2)

            # Resposta
            pdf.set_font("Arial", "", 10)
            pdf.multi_cell(0, 6, f"Resposta: {resposta}")
            pdf.ln(8)

        pdf.output(caminho_pdf)
        return f"✅ PDF gerado: {caminho_pdf}"

    except Exception as e:
        return f"❌ Erro ao gerar PDF: {str(e)}"


print("✅ Funções auxiliares do Gradio corrigidas!")

✅ Funções auxiliares do Gradio corrigidas!


9.1 Gradio

In [34]:
# --- Interface Gradio Corrigida ---
import gradio as gr

# --- Interface Gradio ---
with gr.Blocks(title="Pandas CSV Analyzer", theme=gr.themes.Soft()) as app:
    gr.Markdown("""
    # 🐼 Consultas em CSV com Pandas via Workflow

    Faça upload de um arquivo CSV e faça perguntas sobre os dados em linguagem natural!
    """)

    with gr.Row():
        with gr.Column(scale=2):
            # Upload CSV
            input_arquivo = gr.File(
                file_count="single",
                type="filepath",
                label="📁 Upload do arquivo CSV",
                file_types=[".csv"]
            )
            upload_status = gr.Textbox(
                label="📊 Status do Upload",
                interactive=False,
                placeholder="Nenhum arquivo carregado ainda..."
            )

        with gr.Column(scale=1):
            # Informações do dataset
            gr.Markdown("### ℹ️ Informações do Dataset")
            gr.Markdown("O status do upload aparecerá ao lado.")

    gr.Markdown("---")

    with gr.Row():
        with gr.Column(scale=3):
            # Entrada de pergunta
            input_pergunta = gr.Textbox(
                label="❓ Digite sua pergunta sobre os dados",
                placeholder="Ex: Qual a média da coluna idade? Quantas linhas tem o dataset?",
                lines=2
            )

        with gr.Column(scale=1):
            # Botões
            botao_enviar = gr.Button("🚀 Enviar Pergunta", variant="primary", size="lg")
            botao_limpar = gr.Button("🗑️ Limpar", variant="secondary")

    # Saída da resposta
    output_resposta = gr.Textbox(
        label="💬 Resposta",
        lines=5,
        interactive=False,
        placeholder="A resposta aparecerá aqui após enviar uma pergunta..."
    )

    gr.Markdown("---")

    with gr.Row():
        with gr.Column():
            # Histórico e PDF
            gr.Markdown("### 📋 Histórico e Relatórios")
            botao_pdf = gr.Button("📄 Gerar Relatório PDF", variant="secondary")
            output_pdf = gr.Textbox(
                label="📁 Arquivo PDF",
                interactive=False,
                placeholder="Nenhum PDF gerado ainda..."
            )

    # Estados (invisíveis)
    df_estado = gr.State(value=None)
    historico_estado = gr.State(value=Historico())

    # --- Conexões dos Eventos ---

    # Upload do arquivo
    input_arquivo.change(
        fn=carregar_dados,
        inputs=[input_arquivo, df_estado],
        outputs=[upload_status, df_estado]
    )

    # Envio da pergunta
    botao_enviar.click(
        fn=processar_pergunta,
        inputs=[input_pergunta, df_estado, historico_estado],
        outputs=[output_resposta, historico_estado]
    )

    # Limpar campo de pergunta
    botao_limpar.click(
        fn=lambda: "",
        outputs=[input_pergunta]
    )

    # Gerar PDF
    botao_pdf.click(
        fn=gerar_pdf,
        inputs=[historico_estado],
        outputs=[output_pdf]
    )

    # Enter para enviar
    input_pergunta.submit(
        fn=processar_pergunta,
        inputs=[input_pergunta, df_estado, historico_estado],
        outputs=[output_resposta, historico_estado]
    )

print("✅ Interface Gradio configurada e pronta!")

# Lança a interface
if __name__ == "__main__":
    app.launch(
        debug=True,
        share=False,  # Mude para True se quiser compartilhar publicamente
        server_name="0.0.0.0",  # Para funcionar no Colab
        server_port=7860
    )

✅ Interface Gradio configurada e pronta!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

 Iniciando processamento: Qual é a quantidade de registros no arquivo?
⚠️ Workflow falhou: The following services are not available: ctx
❌ Todas as tentativas falharam: The following services are not available: ctx
🔄 Tentando processamento direto...
 Processamento direto iniciado...
✅ Código: df.shape[0]
✅ Resultado: 1000
 Iniciando processamento: Qual é a quantidade de registros no arquivo?
⚠️ Workflow falhou: The following services are not available: ctx
❌ Todas as tentativas falharam: The following services are not available: ctx
🔄 Tentando processamento direto...
 Processamento direto iniciado...
✅ Código: df.shape[0]
✅ Resultado: 1000
Keyboard interruption in main thread... closing server.


##10 Teste Rápido

In [13]:
# Teste rápido para verificar se tudo está funcionando
query_teste = "Qual é a avaliação média de cada filial?"
result = await executar_consulta(query_teste)

if result:
    print("\n Sistema funcionando perfeitamente!")
else:
    print("\n Houve algum problema - verifique a configuração da API")

TypeError: executar_consulta() missing 1 required positional argument: 'df_local'

##11 Exemplos de Uso Individual

In [ ]:
# Execute uma consulta por vez para testar

# Exemplo 1
query = "Quais são os 5 produtos mais vendidos?"
result = await executar_consulta(query)

In [ ]:
# Exemplo 2
query = "Qual o total de vendas por região?"
result = await executar_consulta(query)

In [ ]:
# Exemplo 3
query = "Qual produto tem a maior margem de lucro?"
result = await executar_consulta(query)

In [ ]:
# Exemplo 4
query = "Quantas vendas foram feitas em cada mês?"
result = await executar_consulta(query)

In [ ]:
query = "Qual o genêro que mais consome e o segmento do produto?"
result = await executar_consulta(query)

##12 Bateria de Testes

In [ ]:
# Executar múltiplas consultas de uma vez
consultas_exemplo = [
    "Qual é a avaliação média de cada filial?",
    "Quais são os 5 produtos mais vendidos?",
    "Qual o total de vendas por região?",
    "Qual produto tem a maior margem de lucro?",
    "Quantas vendas foram feitas em cada mês?",
    "Qual é a receita total por categoria de produto?",
    "Qual filial teve melhor desempenho?",
    "Mostre a distribuição de preços dos produtos"
]

print(" Executando bateria de testes...")
resultados = await executar_multiplas_consultas(consultas_exemplo)

print(f"\n✅ Concluído! {len([r for r in resultados if r])} consultas processadas com sucesso.")

##13 Modo Interativo

In [ ]:
# Modo interativo - digite suas próprias consultas
print(" MODO INTERATIVO")
print("Digite suas consultas (digite 'sair' para terminar)")
print("-" * 50)

while True:
    try:
        query = input("\n Sua consulta: ").strip()

        if query.lower() in ['sair', 'exit', 'quit', '']:
            print(" Até logo!")
            break

        await executar_consulta(query, mostrar_detalhes=False)

    except KeyboardInterrupt:
        print("\n Interrompido pelo usuário!")
        break
    except Exception as e:
        print(f" Erro: {e}")

##14 Análise Manual dos Dados


In [ ]:
# Análise dos dados originais para comparação
print(" ANÁLISE MANUAL DOS DADOS (para comparação)")
print("="*50)

print(f"Estatísticas básicas:")
print(df.describe())

print(f"\nContagem por categoria:")
for col in df.select_dtypes(include=['object']).columns:
    print(f"\n{col}:")
    print(df[col].value_counts().head())

print(f"\nValores nulos:")
print(df.isnull().sum())

#Dicas e Informações

In [ ]:
print("💡 DICAS PARA USO:")
print("="*50)
print("1. Para consultas complexas, seja específico")
print("2. Use nomes exatos das colunas mostradas na análise")
print("3. Experimente diferentes tipos de análise:")
print("   - Agrupamentos: 'média por categoria'")
print("   - Rankings: 'top 10 produtos'")
print("   - Filtros: 'vendas acima de X valor'")
print("   - Séries temporais: 'vendas por mês'")
print("4. O sistema mostra o código Pandas usado!")

print(f"\n INFORMAÇÕES DO DATASET:")
print(f"Suas colunas: {list(df.columns)}")
print(f"Quantidade de dados: {df.shape[0]} linhas x {df.shape[1]} colunas")
print(f"Tipos de dados: {df.dtypes.value_counts().to_dict()}")

print(f"\n SUGESTÕES DE CONSULTAS:")
example_queries = [
    "Qual a correlação entre preço e quantidade vendida?",
    "Mostre a distribuição de vendas por dia da semana",
    "Qual filial tem o melhor desempenho?",
    "Compare as vendas do primeiro e último trimestre",
    "Identifique produtos com baixo estoque",
    "Qual é o produto mais caro por categoria?",
    "Mostre as vendas médias por vendedor",
    "Qual região tem menor variação de preços?"
]

for i, query in enumerate(example_queries, 1):
    print(f"{i}. {query}")

print(f"\n MODELO UTILIZADO: {config.model}")
print(f" URL DOS DADOS: {config.data_url}")

#Notas de Uso:

*   Configuração da API: Configure sua chave Groq nos Secrets do Colab
*   Execução sequencial: Execute as células na ordem
*   Personalização: Modifique a URL dos dados na Célula 6
*   Debug: Os logs mostram o código Pandas gerado
*   Interativo: Use a Célula 12 para consultas livres

##Estrutura final:

*   Instalação automática de dependências
* Configuração completa da API
* Exploração automática dos dados
* Sistema de workflow robusto
* Exemplos prontos para uso
* Modo interativo
* Sistema de debug integrado


>Desenvolvido por Yuri Arduino Bernardineli Alves | GitHub: [YuriArduino](https://github.com/YuriArduino) | E-mail: yuriarduino@gmail.com